In [36]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
path='C:/Projects/Hackerearth/Problem 2/'
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV 
from sklearn import ensemble,linear_model

In [37]:
data=pd.read_csv(path+'new_trains.csv')
test_data=pd.read_csv(path+'new_test.csv')

In [38]:
data.shape,test_data.shape

((200000, 210), (100000, 209))

In [39]:
data.head()

,segment,no_of_Cities,no_of_dow,no_of_tod,no_of_genre,no_of_title,total_time_spent,dow_1,dow_1_percent,dow_2,...,genres_Swimming_present,genres_Formula1_present,genres_Family_present,genres_Badminton_present,genres_Sport_present,average_per_day,average_per_city,average_per_time,average_per_title,average_per_genre
0,0.0,2,7,14,3,60,87386,3412,0.039045,1737,...,0,0,0,0,0,12483.714286,43693.000000,6241.857143,1456.433333,29128.666667
1,0.0,3,7,16,2,70,16371,5745,0.350925,3346,...,0,0,0,0,0,2338.714286,5457.000000,1023.187500,233.871429,8185.500000
2,0.0,1,1,3,2,2,4142,0,0.000000,0,...,0,0,0,0,0,4142.000000,4142.000000,1380.666667,2071.000000,2071.000000
3,0.0,3,5,6,2,8,8348,658,0.078821,0,...,0,0,0,0,0,1669.600000,2782.666667,1391.333333,1043.500000,4174.000000
4,0.0,2,5,9,3,11,10132,1641,0.161962,480,...,0,0,0,0,0,2026.400000,5066.000000,1125.777778,921.090909,3377.333333


In [40]:
data_copy=data.copy()

In [41]:
del data_copy['segment']

In [42]:
target=data['segment']

In [43]:
from sklearn import cross_validation

In [44]:
tainX,testX,train_y,testy=cross_validation.train_test_split(data_copy,target,test_size=.3)
tainX.shape,train_y.shape,testX.shape,testy.shape

((140000, 209), (140000L,), (60000, 209), (60000L,))

In [45]:
train_batch=xgb.DMatrix( tainX, label=train_y)
val_batch=xgb.DMatrix( testX, label=testy)

In [ ]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic','eval_metric':'auc' }
watchlist  = [ (train_batch,'train'),(val_batch,'eval')]
model = xgb.train(param, train_batch, num_rounds,watchlist,early_stopping_rounds=50) 

In [13]:
param_test1 = {
 'max_depth':range(6,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(tainX,train_y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.81220, std: 0.00279, params: {'max_depth': 6, 'min_child_weight': 1},
  mean: 0.81175, std: 0.00236, params: {'max_depth': 6, 'min_child_weight': 3},
  mean: 0.81208, std: 0.00231, params: {'max_depth': 6, 'min_child_weight': 5},
  mean: 0.80836, std: 0.00247, params: {'max_depth': 8, 'min_child_weight': 1},
  mean: 0.80892, std: 0.00249, params: {'max_depth': 8, 'min_child_weight': 3},
  mean: 0.80957, std: 0.00251, params: {'max_depth': 8, 'min_child_weight': 5}],
 {'max_depth': 6, 'min_child_weight': 1},
 0.8121988086317524)

In [15]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=6,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(tainX,train_y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.81220, std: 0.00279, params: {'gamma': 0.0},
  mean: 0.81218, std: 0.00233, params: {'gamma': 0.1},
  mean: 0.81209, std: 0.00199, params: {'gamma': 0.2},
  mean: 0.81182, std: 0.00255, params: {'gamma': 0.3},
  mean: 0.81180, std: 0.00255, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.8121988086317524)

In [28]:
num_rounds=500

In [29]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic','eval_metric':'auc',
        'learning_rate' :0.1, 'n_estimators':140, 'max_depth':6,
 'min_child_weight':1, 'gamma':0, 'subsample':0.8, 'colsample_bytree':0.8,
 'objective': 'binary:logistic', 'nthread':4, 'scale_pos_weight':1,'seed':27}
watchlist  = [ (train_batch,'train'),(val_batch,'eval')]
model = xgb.train(param, train_batch, num_rounds,watchlist,early_stopping_rounds=50) 

[0]	train-auc:0.798672	eval-auc:0.786935
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.805315	eval-auc:0.790946
[2]	train-auc:0.811007	eval-auc:0.79578
[3]	train-auc:0.812537	eval-auc:0.796701
[4]	train-auc:0.81419	eval-auc:0.797645
[5]	train-auc:0.814959	eval-auc:0.798619
[6]	train-auc:0.815587	eval-auc:0.799347
[7]	train-auc:0.815781	eval-auc:0.799687
[8]	train-auc:0.816468	eval-auc:0.800191
[9]	train-auc:0.816938	eval-auc:0.800701
[10]	train-auc:0.817362	eval-auc:0.801144
[11]	train-auc:0.818023	eval-auc:0.80162
[12]	train-auc:0.818572	eval-auc:0.801963
[13]	train-auc:0.819633	eval-auc:0.802443
[14]	train-auc:0.820267	eval-auc:0.802745
[15]	train-auc:0.821069	eval-auc:0.803109
[16]	train-auc:0.821846	eval-auc:0.803304
[17]	train-auc:0.822569	eval-auc:0.803792
[18]	train-auc:0.823199	eval-auc:0.803887
[19]	train-auc:0.823834	eval-auc:0.803893
[20]	train-auc:0.824721	eval-auc

In [30]:
model = xgb.train(param, train_batch, 74,watchlist,early_stopping_rounds=50) 

[0]	train-auc:0.798672	eval-auc:0.786935
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.805315	eval-auc:0.790946
[2]	train-auc:0.811007	eval-auc:0.79578
[3]	train-auc:0.812537	eval-auc:0.796701
[4]	train-auc:0.81419	eval-auc:0.797645
[5]	train-auc:0.814959	eval-auc:0.798619
[6]	train-auc:0.815587	eval-auc:0.799347
[7]	train-auc:0.815781	eval-auc:0.799687
[8]	train-auc:0.816468	eval-auc:0.800191
[9]	train-auc:0.816938	eval-auc:0.800701
[10]	train-auc:0.817362	eval-auc:0.801144
[11]	train-auc:0.818023	eval-auc:0.80162
[12]	train-auc:0.818572	eval-auc:0.801963
[13]	train-auc:0.819633	eval-auc:0.802443
[14]	train-auc:0.820267	eval-auc:0.802745
[15]	train-auc:0.821069	eval-auc:0.803109
[16]	train-auc:0.821846	eval-auc:0.803304
[17]	train-auc:0.822569	eval-auc:0.803792
[18]	train-auc:0.823199	eval-auc:0.803887
[19]	train-auc:0.823834	eval-auc:0.803893
[20]	train-auc:0.824721	eval-auc

In [32]:
ped2=model.predict(val_batch)

In [33]:
ped2

array([ 0.01060944,  0.01228522,  0.01565886, ...,  0.00695435,
        0.13187721,  0.02882161], dtype=float32)

In [34]:
from sklearn import metrics

In [35]:
metrics.roc_auc_score(testy,ped2)

0.80959007437595554

In [ ]:
pred=xgb1.predict_proba(testX.values)

In [ ]:
lala=pd.read_csv('prob2xgb.csv')

In [ ]:
sub=pd.concat([lala['ID'],pd.DataFrame(pred[::,1])],axis=1)
sub.columns=['ID','segment']
sub.to_csv('xgbcv5.csv',index=False)